* meta-algorithm : 한사람의 의견을 듣기보다 여러사람의 의견을 들어 종합해보자 

* classification imbalance : target 값이 true 인것과 false 인것의 데이터의 격차가 큰 경우 문제


# 7.1 Classifiers using multiple samples of the dataset

* AdaBoost 
    - 장점 : 오류율이 낮고 코드가 쉬움, 가장 좋은 약한 분류기를 가지고 분류, 조절을 위한 매개변수가 없음
    - 단점 : Outliers 에 민감
    - 활용 : 수치형 값, 명목형 값
    
* 앙상블 메소드 (ensemble method 또는 meta algorithm) : 서로다른 알고리즘 형태로 사용, 설정다르게 또는 데이터 집합의 서로 다른 부분을 분류기에 적용


## 7.1.1 Building classifiers from randomly resampled data: bagging

* **bagging** : 다양한 분류기를 통합하는 방법, 원본 데이터 집합으로부터 S 개의 새로운 데이터 집합을 만듬 (새로운 데이터 집합의 크기는 원본 데이터 집합의 크기와 같음) 

S 번 만들어진 각각의 데이터 집합은 원본을 대신하기 위해, 원본으로부터 임의로 선택된 예제들로 구축, 두번 이상 같은 예제를 선택할 수도 있고 새로운 집합에 없을 수도 있다.  

S 개의 약한 분류기를 새로운 데이터에 적용하고 다수결로 결정 

* 랜덤포레스트 : 여러 개의 결정 트리들을 임의적으로 학습하는 방식의 앙상블 방법, 랜덤 포레스트 방법은 크게 다수의 결정 트리를 구성하는 학습 단계와 입력 벡터가 들어왔을 때, 분류하거나 예측하는 테스트 단계로 구성되어있다. 랜덤 포레스트는 검출, 분류, 그리고 회귀 등 다양한 애플리케이션으로 활용 by wikipedia

## 7.1.2 Boosting

* **boosting** : 배깅은 언제나 동일한 유형의 분류기를 사용하는데 비해 부스팅은 순차적으로 다른 유형의 약한 분류기를 사용, 이전 분류기에서 잘못 분류된 데이터에 초점을 맞춰 새로운 분류기를 만듬 

* AdaBoost 접근법
    - 준비 : 모든 유형의 데이터 처리, 약한 학습기 사용
    - 학습 : 동일한 데이터 집합으로 약한 학습기를 여러번 훈련시킴 
    - 검사 : 오류율
    - 사용 : 두가지 분류 항목중 하나를 예측, 2 개 이상의 데이터 분류에는 같은 방버을 여러번 적용해야 함 
    
    
# 7.2 Train: improving the classifier by focusing on errors
    
* 분류기 
    - 약함: 분류기가 임의로 추축하는 것보다 일을 잘 처리 (분류 항복이 두개인 경우 오류율이 50% 보다 높다.) 
    - 강함: 낮은 오류율 

* AdaBoost (adaptive boosting) : 가중치는 훈련 데이터에 있는 모든 예제에 적용 
초기에는 가중치들은 모두 동일, 훈련데이터로 훈련 후 오류를 계산, 약한 분류기는 동일한 데이터 집합을 가지고 두 번째 훈련을 시도함. 두번쨰 약한 분류기의 훈련은 훈련 집합의 가중치를 적응하게 됨. 

첫번째에서 이미 분류된 예제들은 가중치를 낮게 부여하고 확실하게 분류되지 못한 예제들은 더 높은 가중치를 부여함 

adaboost 는 모든 약한 분류기에 a(알파) 값을 부여해 오류를 기반으로 답을 구함 

$e = \frac{잘못분류된데이터}{전체데이터}$
    
$a= \frac{1}{2} ln( \frac{1-e}{e} )$

a 를 계산 후, 가중치 벡터 d 를 갱신 

* 확실하게 예측한 경우 

D(t+1) = $\frac{D(t) e^{-a}}{Sum(D)}$

* 예측하지 못한 경우 

D(t+1) = $\frac{D(t) e^{a}}{Sum(D)}$

D 가 계단되고 나면 AdaBoost 는 반복함 훈련 오류가 0이 되거나, 약한 분류기들의 수가 사용자가 정한 값에 도달할 때까지 훈련과 가중치 조절을 되풀이함 weight-adjusting


# 7.3 의사결정 스텀프로 약한 학습기 생성


** decision stump ** 의사결정 트리, 하나의 속성에 대한 하나의 의사결정을 만들고 하나의 분할을 가지는 트리 (= 하나의 스텀프) 구현


In [3]:
import numpy as np
def loadSimpData():
    datMat = np.matrix([[ 1. ,  2.1],
        [ 2. ,  1.1],
        [ 1.3,  1. ],
        [ 1. ,  1. ],
        [ 2. ,  1. ]])
    classLabels = [1.0, 1.0, -1.0, -1.0, 1.0]
    return datMat,classLabels

In [4]:
datMat, classLabels = loadSimpData()

1. 어떤 값이 테스트를 위한 임계 값보다 작거나 큰 경우, 값을 검사에 사용
2. 관련이 더 많은 함수는 데이터 집합에 가중치가 부여된 상태로 반복
3. 가장 낮은 오류의 스텀프를 찾음

의사코드

    minError 설정
    for 데이터집합에 있는 속성을 반복 
        for 모든 단계를 반복 
            for 부등호를 반복
                의사결정 스텀프를 구축, 가중치가 부여된 데이터 집합을 검사 
                오류가 minError 보다 작으면, 스텀프를 저장 
    가장 좋은 스텀프 반환




In [6]:
# 데이터를 분류하기위한 임계값 비교 
def stumpClassify(dataMatrix, dimen, threshVal, threshIneq):
    retArray = np.ones((np.shape(dataMatrix)[0], 1))
    if threshIneq == 'lt':
        retArray[dataMatrix[:,dimen] <= threshVal] = -1.0
    else:
        retArray[dataMatrix[:,dimen] > threshVal] = -1.0
    return retArray

배열 필터링 사용 

데이터 분류 항목을 -1로 처리하고 다른 한쪽의 데이터를 1로 처리, 데이터 집합에 있는 일부 속성과 비교 할 수 있고, 크다, 작다로 전환할 수 있다. 

In [72]:
def buildStump(dataArr, classLabels, D):
    dataMatrix = np.mat(dataArr); labelMat = np.mat(classLabels).T
    m,n = np.shape(dataMatrix)
    
    # 데이터 집합에서 가장 좋은 의사결정 스텀프를 찾아 데이터 가중치 벡터 D에 보관 
    # bestStump 에는 의사결정 스텀프 중 가장 좋은 선택에 해당하는 분류기의 정보를 저장
    numSteps = 10.0; bestStump = {}; bestClasEst = np.mat(np.zeros((m,1)))
    minError = np.inf #init error sum, to +infinity
    
    for i in range(n): #loop over all dimensions
        rangeMin = dataMatrix[:,i].min(); rangeMax = dataMatrix[:,i].max()
    
        # 속성의 수치형 값을 고려해 stepsize 가 얼마나 큰지 확인 
        stepSize = (rangeMax-rangeMin)/numSteps
        
        #loop over all range in current dimension
        for j in range(-1,int(numSteps)+1):
        
            #go over less than and greater than
            for inequal in ['lt', 'gt'] :
                # 양쪽 범위 바깥쪽의 임계값을 설정 
                threshVal = (rangeMin + float(j) * stepSize)
                
                #print(dataMatrix[:,i])
                #print(i, j, inequal,":", stepSize, 'th',threshVal)
                
                # 3개의 순환 변수 기반으로 예측된 분류 항목을 반환 
                predictedVals = stumpClassify(dataMatrix,i,threshVal,inequal)
                #print(predictedVals)
                
                # 1은 labelMat 과 동일하지 않는 경우 부여(오류)
                errArr = np.mat(np.ones((m,1)))
                errArr[predictedVals==labelMat]=0
                
                # 오류에 가중치 D 를 곱하고 단일값을 얻기위해 합함
                # 분류기와 상호작용 가중치 D를 기반으로 분류기를 평가 다른 오류 측정은 사용하지 않음
                weightedError = D.T*errArr
                #print("e", errArr)
                #print("w",weightedError)
                
                #print("split: dim %d, thresh %.2f, thresh ineqal: %s, the weighted error is %.3f" % (i, threshVal, inequal, weightedError)) 
                if weightedError < minError:
                    minError = weightedError
                    bestClasEst = predictedVals.copy()
                    bestStump['dim'] = i
                    bestStump['thresh'] = threshVal
                    bestStump['ineq'] = inequal
                #print(bestStump)
                    
    return bestStump,minError,bestClasEst

In [73]:
D = np.mat(np.ones((5,1))/5)
D

matrix([[ 0.2],
        [ 0.2],
        [ 0.2],
        [ 0.2],
        [ 0.2]])

In [74]:
np.shape(datMat)

(5, 2)

In [75]:
for d, a in zip(datMat, classLabels):
    print(d, a)

[[ 1.   2.1]] 1.0
[[ 2.   1.1]] 1.0
[[ 1.3  1. ]] -1.0
[[ 1.  1.]] -1.0
[[ 2.  1.]] 1.0


In [76]:
buildStump(datMat, classLabels, D)

({'dim': 0, 'ineq': 'lt', 'thresh': 1.3}, matrix([[ 0.2]]), array([[-1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.]]))

모두 똑같은 D 값을 사용하여 첫번째 데이터가 잘못분류됨 

# 7.4 Implementing the full AdaBoost algorithm

의사 코드
    
    for 반복문 반복
        buildStump() 를 사용해 좋은 스텀프를 찾음
        스텀프배열에 가장 좋은 스텀프 추가
        알파 계산
        새로운 가중치 벡터 D 계산
        집계된 분류 항목의 추정치 갱신
        if 오류율이 0.0 이면 for 문을 멈춤 
        
   

In [92]:
# 입력 데이터, 분류항목표시, 반복횟수 
def adaBoostTrainDS(dataArr,classLabels,numIt=40):
    # 약한 분류기 배열 
    weakClassArr = []
    m = np.shape(dataArr)[0]
    
    # 컬럼벡터 : 가중치 초기화 (분류가 잘못된 부분에 가중치를 높임)
    # 확률 벡터 임으로 모든 원소를 합하면 1이 됨 1/m 초기화하는 이유 
    D = np.mat(np.ones((m,1))/m)
        
    # 모든 데이터의 분류 항목을 예측한 집합을 제공
    aggClassEst = np.mat(np.zeros((m,1)))
    
    for i in range(numIt):
        # 의사 결정 스텀프 구축 D 를 사용해 
        # 오류가 적은 스텀프, 오류값과 예측된 분류 항목 벡터 반환
        bestStump, error, classEst = buildStump(dataArr, classLabels, D)
        
        # 현재 스텀프의 결과에 부여할 가중치 
        # max(error, 1e-16) 0으로 나누어 발생하는 오류 확인 
        alpha = float(0.5 * np.log((1.0-error)/max(error, 1e-16)))
        
        # 분류에 필요한 모든것을 저장하는 딕셔너리 
        bestStump['alpha'] = alpha 
        weakClassArr.append(bestStump)  
        
        # 다음 반복을 위한 가중치 계산 
        expon = np.multiply(-1*alpha*np.mat(classLabels).T, classEst)
        D = np.multiply(D, np.exp(expon))
        D = D/D.sum()
        
        # 예측된 분류 항목의 함계를 유지 
        aggClassEst += alpha*classEst
        # sign() 이진 형의 분류 항목을 구함
        aggErrors = np.multiply(np.sign(aggClassEst) != np.mat(classLabels).T,np.ones((m,1)))
        errorRate = aggErrors.sum()/m
        print("total error: ",errorRate)
        
        if errorRate == 0.0: break
    return weakClassArr

In [93]:
np.sign([1, 12, -12, 0])

array([ 1,  1, -1,  0])

aggClassEst 의 부호를 확인하면 전체 분류 항목을 확인 할 수 있다.

In [94]:
classifierArray = adaBoostTrainDS(datMat, classLabels, 9)
classifierArray

total error:  0.2
total error:  0.2
total error:  0.0


[{'alpha': 0.6931471805599453, 'dim': 0, 'ineq': 'lt', 'thresh': 1.3},
 {'alpha': 0.9729550745276565, 'dim': 1, 'ineq': 'lt', 'thresh': 1.0},
 {'alpha': 0.8958797346140273,
  'dim': 0,
  'ineq': 'lt',
  'thresh': 0.90000000000000002}]

# 7.5 Test: classifying with AdaBoost

분류기가 학습, 적용 시킴 

약한 분류기의 결과는 각각에 해당하|는 알파에 의해 가중치가 부여되고, 모든 약한 분류기로부터 가중치가 부여된 겨로가들을 더함. 마지막 합계의 부호를 처리.



In [104]:
# 약한 분류기의 훈련으로 분류를 수행 
# datToClass 데이터 사례들을 곱하너가 분류하는 상요 
# classifierArr 약한 분류기 배열 
def adaClassify(datToClass,classifierArr):
    dataMatrix = np.mat(datToClass)#do stuff similar to last aggClassEst in adaBoostTrainDS
    m = np.shape(dataMatrix)[0]
    # adaBoostTrainDS() 와 같음 
    aggClassEst = np.mat(np.zeros((m,1)))
    
    # 모든 약분류기를 확인 
    for i in range(len(classifierArr)):
        # 분류항목을 예측 
        # 가능한 스텀프의 값을 반복하여, 가중치가 부여된 가장 작은 오류를 가진 스텀프를 선택
        classEst = stumpClassify(dataMatrix,classifierArr[i]['dim'],\
                                 classifierArr[i]['thresh'],\
                                 classifierArr[i]['ineq'])#call stump classify
        
        # 분류항목에 대한 예측 값을 각각의 스텀프에 대한 알파값과 곱함
        aggClassEst += classifierArr[i]['alpha']*classEst
        print(aggClassEst)
        
    #부호 반환
    return np.sign(aggClassEst)

In [105]:
classifierArr = adaBoostTrainDS(datMat, classLabels, 30)
classifierArr

total error:  0.2
total error:  0.2
total error:  0.0


[{'alpha': 0.6931471805599453, 'dim': 0, 'ineq': 'lt', 'thresh': 1.3},
 {'alpha': 0.9729550745276565, 'dim': 1, 'ineq': 'lt', 'thresh': 1.0},
 {'alpha': 0.8958797346140273,
  'dim': 0,
  'ineq': 'lt',
  'thresh': 0.90000000000000002}]

In [106]:
adaClassify([[5,5], [0,0]], classifierArr)

[[ 0.69314718]
 [-0.69314718]]
[[ 1.66610226]
 [-1.66610226]]
[[ 2.56198199]
 [-2.56198199]]


matrix([[ 1.],
        [-1.]])

# 7.6 Example: AdaBoost on a difficult dataset

말의 배앓이에 대한 데이터로 로지스틱 회귀를 사용하여 말이 살젃인지 죽을 것인지 예측 


In [107]:
def loadDataSet(fileName):      #general function to parse tab -delimited floats
    numFeat = len(open(fileName).readline().split('\t')) #get number of fields 
    dataMat = []; labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr =[]
        curLine = line.strip().split('\t')
        for i in range(numFeat-1):
            lineArr.append(float(curLine[i]))
        dataMat.append(lineArr)
        labelMat.append(float(curLine[-1]))
    return dataMat,labelMat

In [108]:
datArr, labelArr = loadDataSet('data/horseColicTraining2.txt')
classifierArray = adaBoostTrainDS(datArr, labelArr, 10)
classifierArr

total error:  0.284280936455
total error:  0.284280936455
total error:  0.247491638796
total error:  0.247491638796
total error:  0.254180602007
total error:  0.240802675585
total error:  0.240802675585
total error:  0.220735785953
total error:  0.247491638796
total error:  0.230769230769


[{'alpha': 0.6931471805599453, 'dim': 0, 'ineq': 'lt', 'thresh': 1.3},
 {'alpha': 0.9729550745276565, 'dim': 1, 'ineq': 'lt', 'thresh': 1.0},
 {'alpha': 0.8958797346140273,
  'dim': 0,
  'ineq': 'lt',
  'thresh': 0.90000000000000002}]

In [112]:
testArr, testLabelArr = loadDataSet('data/horseColicTest2.txt')
prediction10 = adaClassify(testArr, classifierArr)
prediction10

[[ 0.69314718]
 [ 0.69314718]
 [-0.69314718]
 [-0.69314718]
 [ 0.69314718]
 [-0.69314718]
 [ 0.69314718]
 [ 0.69314718]
 [ 0.69314718]
 [ 0.69314718]
 [ 0.69314718]
 [-0.69314718]
 [-0.69314718]
 [ 0.69314718]
 [ 0.69314718]
 [-0.69314718]
 [ 0.69314718]
 [-0.69314718]
 [ 0.69314718]
 [-0.69314718]
 [-0.69314718]
 [ 0.69314718]
 [-0.69314718]
 [-0.69314718]
 [ 0.69314718]
 [ 0.69314718]
 [ 0.69314718]
 [-0.69314718]
 [ 0.69314718]
 [-0.69314718]
 [ 0.69314718]
 [-0.69314718]
 [-0.69314718]
 [ 0.69314718]
 [-0.69314718]
 [-0.69314718]
 [-0.69314718]
 [ 0.69314718]
 [ 0.69314718]
 [ 0.69314718]
 [ 0.69314718]
 [-0.69314718]
 [-0.69314718]
 [ 0.69314718]
 [-0.69314718]
 [ 0.69314718]
 [-0.69314718]
 [-0.69314718]
 [-0.69314718]
 [-0.69314718]
 [-0.69314718]
 [ 0.69314718]
 [-0.69314718]
 [-0.69314718]
 [ 0.69314718]
 [ 0.69314718]
 [-0.69314718]
 [-0.69314718]
 [-0.69314718]
 [-0.69314718]
 [ 0.69314718]
 [-0.69314718]
 [ 0.69314718]
 [ 0.69314718]
 [-0.69314718]
 [ 0.69314718]
 [ 0.69314

matrix([[ 1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.],
        [-1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.],
        [ 1.],
        [-1.],
        [ 1.],
        [-1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [-1.],
        [ 1.],
        [-1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.],
        [-1.],
        [-1.],
        [-1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.],
        [-1.],
        [ 1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [ 1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.],
        [ 1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 1.],
        [-1.],
        [ 1.],
        [ 

In [113]:
errArr = np.mat(np.ones((67,1)))
errArr[prediction10!=np.mat(testLabelArr).T].sum()

28.0

검사오류가 최소에 도달한 후 다시 증가함. (과적합)
plateau 에 도달 이보다 더 많은 분류기를 사용해도 성능이 좋아 지지 않는다. (누락된 값의 정리가 필요하다.)

에이다 부스트와 SVM 은 지도학습에서 성능 좋은 알고리즘이다. <br>
에이다 부스트에서 약한 분류기를 SVM 의 커널처럼 생각 할 수 있다. <br> 
최소마진을 최대화 하는 방식으로 에이다 부스트 알고리즘을 작성할 수도 있다. 


모든 분류기들의 공통적 문제 

# 7.7 Classification imbalance 

## 7.7.1 Alternative performance metrics: precision, recall, and ROC

사례의 횟수를 검사에 사용된 전체 사례의 수로 나눈것은 잘못 분류된 것이 어떤 사례인지 모른다. 

Confusion Matrix, ROC Curve 사용 

## 7.7.2 Manipulating the classifier’s decision with a cost function

비용 민감 학습 (cost-sentitive learning) : 고르지 못한 분류 비용을 활용하여 분류기의 임계값을 조정

분류 알고리즘에 비용 정보를 포함 시키기 위한 방법 
    - AdaBoost 는 비용함수를 기반으로 하는 오류 가중치 벡터 D 를 조절 할 수 있음
    - 나이브베이즈 에서는 가장 높은 확률을 가지는 분류 항목 대신 가장 낮은 비용이 측정되는 분류 항목을 예측하도록 함 
    - SVM 서로 다른 분류 항목에 대한 비용 함수에서 매개 변수 C 를 다르게 하여 사용
    
## 7.7.3 Data sampling for dealing with classification imbalance

- Undersampling : 데이터의 일부를 제거해서 사용 
- Oversampling : 데이터를 중복해서 사용

샘플링은 임의로 선택하거나 미리 결정된 방식으로 선택 

주로 의사결정 범위로부터 떨어져 있는 예제를 제거 하는 방식을 사용 

부정적인 분류 항목에 속하는 데이터를 언더샘플링하는데 있어 Hybrid 방식을 사용하고 긍정적인 분류 항목에 속하는 데이터는 오버샘플링을 함


긍정적인 분류 항목을 오버샘플링하기 위해서는 기존 예제를 복제하거나 기존의 데이터와 유사한 새로운 데이터를 추가하는 방법을 사용할 수 있다. 이렇게 추가된 데이터는 기존 데이터에 의한 결과를 보정한다. 하지만 과적합 될 수 있다. 


배깅 : 대체할 데이터 집합을 위해 원본 데이터 집합과 같은 크기의 데이터 집합을 임의로 추출한 데이터들로 구축

부스팅 : 하나의 데이터 집합에 순차적으로 서로 다른 여러가지 분류기를 적용함으로써 단계를 추가

앙상블 메소드 : 간단하게 하나의 분류기를 사용하는 것보다 더 좋은 답을 구하기 위해서 다양한 분류기들의 예측을 결합하는 방법, 랜덤포레스트를 성공적으로 보여줌 

분류기들이 서로 큰 차이를 보일때 다양한 분류기들을 결합하는 것이 효과적이다. 
